In [1]:
import functools
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
import scipy
from sklearn.model_selection import train_test_split
import time
import tqdm
from anndata import AnnData
import scanpy as sc

In [2]:
# Spatial LDA imports
from spatial_lda.featurization import featurize_tumors
from spatial_lda.featurization import neighborhood_to_marker
from spatial_lda.featurization import make_merged_difference_matrices
import spatial_lda.model
from spatial_lda.visualization import plot_samples_in_a_row
from spatial_lda.visualization import plot_one_tumor_all_topics
from spatial_lda.visualization import plot_one_tumor_topic
from spatial_lda.visualization import plot_topics_heatmap

c:\Users\40868\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set data path
data_path = "D:/Desktop/MGI/CODE/"

In [4]:
with open(data_path + "Data/patient_dfs.pkl", 'rb') as f:
   patient_dfs = pickle.load(f)

for patient_id in patient_dfs.keys():
  df = patient_dfs[patient_id]
  df['combined_cluster_id'] = (df['immune_cluster'].fillna(0) + 
                               (df.cluster_id + 12).fillna(0))
  df.loc[df['combined_cluster_id'] == 0, 'combined_cluster_id'] = None
  df.loc[:, 'is_tumor'] = ~df['isimmune']
  patient_dfs[patient_id] = df

patient_dfs[1]

,Au,Background,Beta catenin,Ca,CD11b,CD11c,CD138,CD16,CD20,CD209,...,Ta,Vimentin,x,y,area,isimmune,immune_cluster,cluster_id,combined_cluster_id,is_tumor
0,3.449781,0.164687,0.0,2.609307,0.0,0.000000,0.0,0.0,0.000000,0.0,...,2.788951,0.000000,36.328395,161.809877,405,False,NaN,0.0,12.0,True
1,3.441198,0.168433,0.0,2.711448,0.0,0.130399,0.0,0.0,0.000000,0.0,...,2.926451,0.800266,37.880769,190.157692,260,True,5.0,NaN,5.0,False
2,1.233291,0.215964,0.0,2.186117,0.0,0.000000,0.0,0.0,0.005882,0.0,...,0.334882,0.290034,35.029412,203.741176,170,False,NaN,0.0,12.0,True
3,2.032927,0.112490,0.0,2.142954,0.0,0.000000,0.0,0.0,0.234216,0.0,...,1.266202,0.000000,34.443636,224.225455,275,True,3.0,NaN,3.0,False
4,1.987532,0.084782,0.0,1.673678,0.0,0.000000,0.0,0.0,0.078988,0.0,...,1.166704,0.022091,44.255814,254.927907,860,False,NaN,0.0,12.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5235,0.204912,0.095095,0.0,1.993685,0.0,0.000000,0.0,0.0,0.664331,0.0,...,0.047601,0.000000,2016.111111,1105.619048,63,True,1.0,NaN,1.0,False
5236,0.734866,0.028165,0.0,1.167104,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.168220,0.000000,2016.014085,1815.042254,71,False,NaN,0.0,12.0,True
5237,0.230107,0.035707,0.0,1.112817,0.0,0.000000,0.0,0.0,1.894672,0.0,...,0.177636,0.000000,2016.482143,1136.232143,56,True,1.0,NaN,1.0,False
5238,1.057317,0.046858,0.0,1.575251,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.352057,0.000000,2016.453125,1738.546875,64,False,NaN,0.0,12.0,True


In [5]:
with open(data_path + "Data/tumor_marker_features.pkl", 'rb') as f:
    tumor_marker_features = pickle.load(f)
tumor_marker_features

,Beta catenin,Ca,CD11b,CD11c,CD138,CD16,CD20,CD209,CD3,CD31,...,P,p53,Pan-Keratin,PD-L1,PD1,phospho-S6,Si,SMA,Ta,Vimentin
"(1, 0)",0,11,0,0,0,0,2,0,0,0,...,10,0,0,0,0,7,7,1,9,2
"(1, 2)",0,12,0,0,0,0,1,0,0,0,...,10,0,0,0,0,5,4,2,7,3
"(1, 4)",0,14,0,0,0,0,3,0,0,0,...,13,0,0,0,0,7,3,2,7,3
"(1, 5)",0,12,0,0,0,0,3,0,0,0,...,11,0,0,0,0,5,2,2,6,3
"(1, 6)",0,11,0,0,0,0,4,0,0,0,...,10,0,0,0,0,5,0,2,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(41, 4559)",0,10,0,0,0,0,0,0,0,0,...,10,0,0,0,0,0,8,0,4,5
"(41, 4560)",0,5,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,5,0,4,2
"(41, 4561)",0,10,0,0,0,0,0,0,0,0,...,9,0,0,0,0,0,9,0,6,7
"(41, 4563)",0,4,0,0,0,0,0,0,0,0,...,4,0,0,0,0,0,4,0,4,3


In [6]:
TRAIN_SIZE_FRACTION = 0.9

all_tumor_idxs = tumor_marker_features.index.map(lambda x: x[0])
_sets = train_test_split(tumor_marker_features, 
                         test_size=1. - TRAIN_SIZE_FRACTION,
                         stratify=all_tumor_idxs)
train_tumor_marker_features, test_tumor_marker_features = _sets
train_difference_matrices = make_merged_difference_matrices(
    train_tumor_marker_features, patient_dfs, 'x', 'y')
tumor_idxs = train_tumor_marker_features.index.map(lambda x: x[0])

In [7]:
data = sc.read_h5ad(data_path + "Data/SS200000116BR_E6.bin200.h5ad")

# annotation data
anno_data = pd.read_csv(data_path + "Output/RCTD_results.csv", index_col=0)
anno_data.index = anno_data.index.rename('ID')
anno_data.reset_index(inplace=True)
anno_data.drop(['x', 'y'], axis=1, inplace=True)
anno_data

,ID,spot_class,first_type,second_type,first_prob,second_prob,final_type
0,27487790701800,doublet_certain,Hepatocyte,Tumor-Cholang,0.363940,0.636060,Tumor-Cholang
1,29205777624400,doublet_certain,Tumor-Cholang,Hepatocyte,0.635929,0.364071,Tumor-Cholang
2,31782757998200,doublet_certain,Tumor-Cholang,Hepatocyte,0.638376,0.361624,Tumor-Cholang
3,32641751459800,doublet_certain,Tumor-Cholang,Hepatocyte,0.723940,0.276060,Tumor-Cholang
4,39513699138800,doublet_certain,Tumor-Cholang,Hepatocyte,0.862991,0.137009,Tumor-Cholang
...,...,...,...,...,...,...,...
10184,6012954233600,singlet,Hepatocyte,Endothelial,0.544437,0.455563,Hepatocyte
10185,2576980383000,singlet,Hepatocyte,Tumor-Cholang,0.536803,0.463197,Hepatocyte
10186,17179869208000,singlet,Hepatocyte,Tumor-Cholang,0.397441,0.602559,Tumor-Cholang
10187,11166914969600,doublet_certain,Hepatocyte,Tumor-Cholang,0.370117,0.629883,Tumor-Cholang


In [8]:
data_obs = data.obs.copy()
data_obs.index = data_obs.index.rename('ID')
data_obs.reset_index(inplace=True)

data_obs['ID'] = data_obs['ID'].astype(str)
anno_data['ID'] = anno_data['ID'].astype(str)

merged_data = pd.merge(data_obs, anno_data, on='ID', how='inner')
merged_data

,ID,orig.ident,x,y,spot_class,first_type,second_type,first_prob,second_prob,final_type
0,27487790701800,sample,6400,7400,doublet_certain,Hepatocyte,Tumor-Cholang,0.363940,0.636060,Tumor-Cholang
1,29205777624400,sample,6800,11600,doublet_certain,Tumor-Cholang,Hepatocyte,0.635929,0.364071,Tumor-Cholang
2,31782757998200,sample,7400,7800,doublet_certain,Tumor-Cholang,Hepatocyte,0.638376,0.361624,Tumor-Cholang
3,32641751459800,sample,7600,10200,doublet_certain,Tumor-Cholang,Hepatocyte,0.723940,0.276060,Tumor-Cholang
4,39513699138800,sample,9200,15600,doublet_certain,Tumor-Cholang,Hepatocyte,0.862991,0.137009,Tumor-Cholang
...,...,...,...,...,...,...,...,...,...,...
10184,6012954233600,sample,1400,19200,singlet,Hepatocyte,Endothelial,0.544437,0.455563,Hepatocyte
10185,2576980383000,sample,600,5400,singlet,Hepatocyte,Tumor-Cholang,0.536803,0.463197,Hepatocyte
10186,17179869208000,sample,4000,24000,singlet,Hepatocyte,Tumor-Cholang,0.397441,0.602559,Tumor-Cholang
10187,11166914969600,sample,2600,0,doublet_certain,Hepatocyte,Tumor-Cholang,0.370117,0.629883,Tumor-Cholang


In [9]:
# Dataframe to Anndata
merged_data_ann = AnnData(obs=merged_data, var=data.var)
merged_data_ann.obs

c:\Users\40868\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,ID,orig.ident,x,y,spot_class,first_type,second_type,first_prob,second_prob,final_type
0,27487790701800,sample,6400,7400,doublet_certain,Hepatocyte,Tumor-Cholang,0.363940,0.636060,Tumor-Cholang
1,29205777624400,sample,6800,11600,doublet_certain,Tumor-Cholang,Hepatocyte,0.635929,0.364071,Tumor-Cholang
2,31782757998200,sample,7400,7800,doublet_certain,Tumor-Cholang,Hepatocyte,0.638376,0.361624,Tumor-Cholang
3,32641751459800,sample,7600,10200,doublet_certain,Tumor-Cholang,Hepatocyte,0.723940,0.276060,Tumor-Cholang
4,39513699138800,sample,9200,15600,doublet_certain,Tumor-Cholang,Hepatocyte,0.862991,0.137009,Tumor-Cholang
...,...,...,...,...,...,...,...,...,...,...
10184,6012954233600,sample,1400,19200,singlet,Hepatocyte,Endothelial,0.544437,0.455563,Hepatocyte
10185,2576980383000,sample,600,5400,singlet,Hepatocyte,Tumor-Cholang,0.536803,0.463197,Hepatocyte
10186,17179869208000,sample,4000,24000,singlet,Hepatocyte,Tumor-Cholang,0.397441,0.602559,Tumor-Cholang
10187,11166914969600,sample,2600,0,doublet_certain,Hepatocyte,Tumor-Cholang,0.370117,0.629883,Tumor-Cholang


In [10]:
aligned_X = data[data.obs.index.isin(merged_data_ann.obs["ID"])].X
print(aligned_X.toarray())

[[  3.   0.   0. ...   5.   5.  66.]
 [  2.   0.   0. ...   4.  12. 110.]
 [  1.   0.   0. ...  24.  14. 167.]
 ...
 [  0.   0.   0. ...   0.   0.  13.]
 [  0.   0.   0. ...   0.   0.   1.]
 [  0.   0.   0. ...   0.   0.   1.]]


In [11]:
merged_data_ann = AnnData(X=aligned_X.toarray(), obs=merged_data, var=data.var)
merged_data_ann

c:\Users\40868\AppData\Local\Programs\Python\Python39\lib\site-packages\anndata\_core\aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AnnData object with n_obs × n_vars = 10189 × 30523
    obs: 'ID', 'orig.ident', 'x', 'y', 'spot_class', 'first_type', 'second_type', 'first_prob', 'second_prob', 'final_type'

In [12]:
X = pd.DataFrame(merged_data_ann.X.astype(int), columns=merged_data_ann.var.index)
X

,AL354822.1,AC233755.2,AC007325.2,AC011841.1,MAFIP,BX004987.1,AC136616.1,AC136612.1,TTTY3B,DAZ2,...,AEBP1,KLHL8,AC237221.1,AC006455.1,SELENOI,HABP2,SEC61G,HSP90B1,IGFBP1,SERPINA1
0,3,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,1,2,5,5,66
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,5,4,12,110
2,1,0,0,0,0,0,0,0,0,0,...,3,0,0,0,1,6,5,24,14,167
3,1,0,0,0,0,0,0,0,0,0,...,3,0,0,0,1,4,2,10,9,99
4,1,0,0,0,0,0,0,0,0,0,...,17,1,0,0,6,25,56,148,44,781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10184,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
10187,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
TRAIN_SIZE_FRACTION = 0.9

_sets = train_test_split(X, test_size=1. - TRAIN_SIZE_FRACTION)

train, test = _sets


In [25]:
from scipy.sparse import coo_matrix
from scipy.sparse.csgraph._min_spanning_tree import minimum_spanning_tree
from scipy.spatial import Voronoi

def make_merged_difference_matrices_re(sample_features, sample_dfs,
                                    x_col, y_col, z_col=None,
                                    reduce_to_mst=True):
    difference_matrices = dict()
    sample_idxs = sample_features.index
    for sample_idx in set(sample_idxs):
        graph = make_nearest_neighbor_graph_re(
            sample_features, sample_dfs, sample_idx, x_col, y_col, z_col=z_col)
        num_nodes, src_nodes, dst_nodes, edge_lengths = graph
        difference_matrix = make_difference_matrix_re(
            num_nodes, src_nodes, dst_nodes)
        if reduce_to_mst:
            mst_mask = make_minimum_spaning_tree_mask_re(
                num_nodes, src_nodes, dst_nodes, edge_lengths)
            difference_matrix = difference_matrix[mst_mask, :]
        difference_matrices[sample_idx] = difference_matrix
    return difference_matrices

def make_difference_matrix_re(num_nodes, src_nodes, dst_nodes):
    num_edges = len(src_nodes)
    rows = np.hstack([np.arange(num_edges), np.arange(num_edges)])
    cols = np.hstack([src_nodes, dst_nodes])
    values = np.hstack([np.ones(num_edges), -1 * np.ones(num_edges)])
    difference_matrix = coo_matrix(
        (values, (rows, cols)), shape=(num_edges, num_nodes)).tocsr()
    return difference_matrix

def make_nearest_neighbor_graph_re(sample_features, sample_dfs, sample_idx, x_col, y_col, z_col=None):
    sample_idxs = sample_features.index
    sample_rows = sample_features[sample_idxs == sample_idx]
    cell_idx = sample_rows.index.map(lambda x: x[1])
    if z_col is None:
        coords = [x_col, y_col]
    else:
        coords = [x_col, y_col, z_col]
    cell_coords = sample_dfs[sample_idx].loc[cell_idx][coords].values
    vor = Voronoi(cell_coords)
    num_edges = vor.ridge_points.shape[0]
    num_nodes = len(cell_coords)
    src_nodes = vor.ridge_points[:, 0]
    dst_nodes = vor.ridge_points[:, 1]
    coord_difference = cell_coords[src_nodes] - cell_coords[dst_nodes]
    edge_lengths = np.sqrt(np.sum(coord_difference**2.0, axis=1))
    assert len(edge_lengths) == num_edges
    return num_nodes, src_nodes, dst_nodes, edge_lengths

def make_minimum_spaning_tree_mask_re(num_nodes, src_nodes, dst_nodes,
                                   edge_lengths):
    num_edges = len(src_nodes)
    adjacency_matrix = coo_matrix((edge_lengths, (src_nodes, dst_nodes)),
                                  shape=(num_nodes, num_nodes)).tocsr()
    edge_index = coo_matrix((np.arange(num_edges), (src_nodes, dst_nodes)),
                            shape=(num_nodes, num_nodes)).tocsr()
    spanning_tree = minimum_spanning_tree(adjacency_matrix)
    mst_mask = np.asarray(edge_index[spanning_tree.nonzero()]).squeeze()
    return mst_mask

train_difference_matrices = make_merged_difference_matrices_re(train, merged_data_ann.obs, 'x', 'y')

TypeError: 'int' object is not subscriptable